## VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.simplefilter('ignore')
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Import .csv from WeatherPy (Part 1) into a DataFrame

In [2]:
source_file = './output_data/output_cities_AR.csv'
cities_df = pd.read_csv(source_file, sep = ",", low_memory=False)
cities_df['Date'] = pd.to_datetime(cities_df['Date'], unit='s')
cities_df.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Qaanaaq,77.48,-69.36,14.79,81,0,7.00,GL,2020-09-29 21:15:01
1,Lata,40.16,-8.33,66.00,88,55,2.06,PT,2020-09-29 21:17:54
2,Hilo,19.73,-155.09,84.20,66,1,12.75,US,2020-09-29 21:17:54
3,Bandarbeyla,9.49,50.81,77.90,81,93,22.73,SO,2020-09-29 21:17:54
4,Hermanus,-34.42,19.23,51.01,86,11,6.76,ZA,2020-09-29 21:17:55


In [3]:
n_rows, n_cols = cities_df.shape
print(f'There are {n_rows} rows and {n_cols} columns.')

There are 558 rows and 9 columns.


In [4]:
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558 entries, 0 to 557
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   City        558 non-null    object        
 1   Latitude    558 non-null    float64       
 2   Longitude   558 non-null    float64       
 3   Max Temp    558 non-null    float64       
 4   Humidity    558 non-null    int64         
 5   Cloudiness  558 non-null    int64         
 6   Wind Speed  558 non-null    float64       
 7   Country     555 non-null    object        
 8   Date        558 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(2), object(2)
memory usage: 39.4+ KB


### Humidity Heat Map

In [5]:
# Configure gmaps w API key
gmaps.configure(api_key = g_key)
locations = cities_df[["Latitude", "Longitude"]]
weights = cities_df['Humidity']

In [6]:
# Use the Lat and Lng as locations and Humidity as the weight.
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Add Heatmap layer to map.
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations,
                                 weights = weights, dissipating = False,
                                 max_intensity = 300, point_radius = 5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create a new Data Frame
Narrow list to include only the cities that meet ALL of the following criteria:
* Max Temperature between 70 - 80 degrees. 
* Wind Speed less than 10 mph.
* Zero Cloudiness 

In [8]:
hotel_df = cities_df.loc[(cities_df['Max Temp'] < 80) & 
                         (cities_df['Max Temp'] > 70) 
                              & (cities_df['Wind Speed'] < 10) & (cities_df['Humidity'] <=50)].copy() # I like clouds

### Hotel Map

In [9]:
# * Store into variable named `hotel_df`.
# Add a "Hotel Name" column to the DataFrame.
hotel_df['Hotel Name'] = ""
hotel_df.reset_index(drop=True).head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Nizwá,22.93,57.53,78.08,33,0,2.08,OM,2020-09-29 21:18:25,
1,Maun,-19.98,23.42,77.00,25,0,8.05,BW,2020-09-29 21:15:17,
2,Dahuk,36.87,42.99,70.95,27,39,3.71,IQ,2020-09-29 21:18:42,
3,Minchianabad,30.16,73.57,79.41,25,0,3.89,PK,2020-09-29 21:18:47,
4,Vallenar,-28.57,-70.76,73.49,26,0,8.72,CL,2020-09-29 21:18:55,


In [10]:
# Set parameters to search for hotels with 5000 meters.
radius = 5000

# Use Google Places API for each city's coordinates.
base_url_nearby = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Store the first Hotel result into the DataFrame.
params = {
    "types": "lodging",
    "radius": radius,
    "key": g_key}

for index,row in hotel_df.iterrows():
    lat=row["Latitude"]
    lng=row["Longitude"]
    params["location"]=f"{lat},{lng}"
    
# RUN REQUEST
    response = requests.get(base_url_nearby, params)
# TEST RESULTS
    results = response.json()
# PRINT RESULTS
    try:
     # record the hotel name
        hotel_df['Hotel Name'][index] = results["results"][0]["name"]
    except IndexError:
        print(f"No hotels available near {hotel_df['City'][index]}.\n")
        hotel_df['Hotel Name'][index] = "none available"
hotel_df

No hotels available near Abu Samrah.



,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
83,Nizwá,22.93,57.53,78.08,33,0,2.08,OM,2020-09-29 21:18:25,Orient Apartments
87,Maun,-19.98,23.42,77.00,25,0,8.05,BW,2020-09-29 21:15:17,Enviro Villa
130,Dahuk,36.87,42.99,70.95,27,39,3.71,IQ,2020-09-29 21:18:42,Parwar Hotel
146,Minchianabad,30.16,73.57,79.41,25,0,3.89,PK,2020-09-29 21:18:47,Ayyaz House
166,Vallenar,-28.57,-70.76,73.49,26,0,8.72,CL,2020-09-29 21:18:55,Humacao Bed & Breakfast
241,San Rafael,-34.62,-68.33,75.20,18,0,7.78,AR,2020-09-29 21:19:28,Hotel San Rafael
380,Najrān,17.49,44.13,77.00,23,31,2.75,SA,2020-09-29 21:20:17,محمد هادي ال فطيح
402,Roebourne,-20.78,117.13,70.16,35,82,6.85,AU,2020-09-29 21:20:25,Harding River Caravan Park
446,Abu Samrah,35.30,37.18,74.61,42,36,8.39,SY,2020-09-29 21:20:40,none available


In [11]:
# Using the template provided, add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [12]:
# Plot markers on top of the heatmap.
new_map = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(new_map)
fig

Figure(layout=FigureLayout(height='420px'))